In [ ]:
%sql
SELECT * FROM df_user;

ind,user_name,age,date_joined
8731,Andrea Alexander,21,2015-11-10T09:27:42.000+0000
4315,Michelle Prince,36,2015-12-20T16:38:13.000+0000
10625,Christian Lang,32,2017-10-10T20:09:33.000+0000
1313,Brittany Jones,32,2016-04-02T03:51:23.000+0000
10794,Thomas Turner,34,2016-12-22T00:02:02.000+0000
2959,David Griffith,20,2016-01-07T19:49:22.000+0000
9875,Brendan Joseph,26,2015-12-20T10:28:00.000+0000
2863,Dylan Holmes,32,2016-10-23T14:06:51.000+0000
5730,Rachel Davis,36,2015-12-08T20:02:43.000+0000
2923,Brian Nelson,26,2015-11-11T03:20:57.000+0000


In [ ]:
from pyspark.sql.functions import udf, col, array, to_timestamp, concat, lit
from pyspark.sql.types import IntegerType, StringType, FloatType
from pyspark.sql import SparkSession

# File location and type
file_location = "/mnt/0a2f66c3e41f.df_user/topics/0a2f66c3e41f.user/partition=0/*.json"
file_type = "json"

# # Ask Spark to infer the schema
infer_schema = "true"

# Read in JSONs from mounted S3 bucket
df = spark.read.format(file_type).option("inferSchema", infer_schema).load(file_location)

# Create a new column user_name that concatenates the information found in the first_name and last_name columns
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 
df_with_fullname = df.withColumn("user_name", concat(col("first_name"),lit(" "), col("last_name")))

# Drop the first_name and last_name columns from the DataFrame
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 
columns_to_drop = ["first_name", "last_name"]
df_with_fullname= df_with_fullname.drop(*columns_to_drop)

# Convert the date_joined column from a string to a timestamp data type
cleaned_df = df_with_fullname.withColumn("date_joined", col("date_joined").cast("timestamp"))

# Reorder the DataFrame columns
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 
cleaned_df = cleaned_df.withColumnRenamed('index', 'ind')
new_order = ["ind", "user_name", "age", "date_joined"]
df_reordered = cleaned_df.select(*new_order)

df_reordered.printSchema()
display(df_reordered)


# Write dataframe to a temporary view
df_reordered.createOrReplaceTempView("0a2f66c3e41f_df_user")

# Create table from temporary view
spark.sql("CREATE TABLE 0a2f66c3e41f_df_user AS SELECT * FROM 0a2f66c3e41f_df_user")


root
-- ind: long (nullable = true)
-- user_name: string (nullable = true)
-- age: long (nullable = true)
-- date_joined: timestamp (nullable = true)

ind,user_name,age,date_joined
8731,Andrea Alexander,21,2015-11-10T09:27:42.000+0000
4315,Michelle Prince,36,2015-12-20T16:38:13.000+0000
10625,Christian Lang,32,2017-10-10T20:09:33.000+0000
1313,Brittany Jones,32,2016-04-02T03:51:23.000+0000
10794,Thomas Turner,34,2016-12-22T00:02:02.000+0000
2959,David Griffith,20,2016-01-07T19:49:22.000+0000
9875,Brendan Joseph,26,2015-12-20T10:28:00.000+0000
2074,Annette Forbes,21,2016-01-03T15:42:12.000+0000
8304,Charles Berry,25,2015-12-28T04:21:39.000+0000
6063,Corey Andrews,23,2015-11-25T13:36:22.000+0000


Out[1]: DataFrame[num_affected_rows: bigint, num_inserted_rows: bigint]